In [1]:
import os
import yt_dlp
import cv2
from ultralytics import YOLO

### 1. Yutube 영상 다운로드

In [2]:
# 다운로드할 YouTube URL
url = "https://www.youtube.com/shorts/ntJl6rIOKOc"

# yt-dlp 설정
ydl_opts = {
    "format": "best",
    "outtmpl": "downloaded_video.%(ext)s",
}

# 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/shorts/ntJl6rIOKOc
[youtube] ntJl6rIOKOc: Downloading webpage
[youtube] ntJl6rIOKOc: Downloading ios player API JSON
[youtube] ntJl6rIOKOc: Downloading mweb player API JSON
[youtube] ntJl6rIOKOc: Downloading player a8476471
[youtube] ntJl6rIOKOc: Downloading m3u8 information
[info] ntJl6rIOKOc: Downloading 1 format(s): 18
[download] Destination: downloaded_video.mp4
[download] 100% of  473.08KiB in 00:00:00 at 5.62MiB/s     


### 2. 동영상의 fps 정보

In [3]:
# 동영상 파일 경로
video_path = "downloaded_video.mp4"

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# FPS 가져오기
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:", fps)

# 동영상 파일 닫기
cap.release()

FPS: 30.0


### 3. 동영상을 프레임 단위로 분할

In [4]:
video_path = "downloaded_video.mp4"

# cv2를 이용해 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 프레임을 저장할 디렉토리 생성
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# 프레임 단위로 이미지로 저장
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 이미지 파일로 프레임 저장
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

# 동영상 파일 닫기
cap.release()
print(f"총 {frame_count}개의 프레임이 저장되었습니다.")

총 152개의 프레임이 저장되었습니다.


### 4. YOLOv8 추론

In [7]:
# Load a model
model = YOLO("yolov8m.pt")  # pretrained YOLO8m model

# "frames" 디렉토리 안에 있는 파일과 폴더 목록을 가져옴
frames = os.listdir("frames/")

# 원하는 디렉토리 경로 설정
directory_path = "results"

# 디렉토리가 없으면 생성
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"디렉토리 '{directory_path}'가 생성되었어.")
else:
    print(f"디렉토리 '{directory_path}'가 이미 존재해.")

for frame in frames:
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model("frames/" + frame, classes=[16, 18], conf=0.6)  # , device="mps" (맥북)

    # Process results list
    for result in results:
        result.save(filename="results/" + frame)  # save to disk

디렉토리 'results'가 생성되었어.

image 1/1 c:\Users\aq348\Desktop\intelligence-1\video_inference\frames\frame_0000.jpg: 640x384 4 sheeps, 616.7ms
Speed: 0.0ms preprocess, 616.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\aq348\Desktop\intelligence-1\video_inference\frames\frame_0001.jpg: 640x384 9 sheeps, 462.7ms
Speed: 1.8ms preprocess, 462.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\aq348\Desktop\intelligence-1\video_inference\frames\frame_0002.jpg: 640x384 6 sheeps, 452.5ms
Speed: 0.0ms preprocess, 452.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\aq348\Desktop\intelligence-1\video_inference\frames\frame_0003.jpg: 640x384 4 sheeps, 660.9ms
Speed: 0.0ms preprocess, 660.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 c:\Users\aq348\Desktop\intelligence-1\video_inference\frames\frame_0004.jpg: 640x384 5 sheeps, 473.3ms
Speed: 0.0ms pr

### 5. 프레임을 동영상으로 변환

In [8]:
import os

# 이미지들이 저장된 폴더 경로와 저장할 비디오 파일 이름 설정
image_folder = 'results'  # 이미지들이 저장된 폴더 경로
video_name = 'output_video.mp4'  # 저장할 비디오 파일 이름

# 이미지 파일 이름을 읽고 정렬 (순서대로 동영상을 만들기 위해)
images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]
images.sort()

# 첫 번째 이미지에서 프레임의 너비와 높이 가져오기
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 비디오 코덱과 파일명 설정, FPS 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4 코덱 설정
video = cv2.VideoWriter(video_name, fourcc, 30, (width, height))  # FPS를 30으로 설정

# 이미지를 순차적으로 불러와서 비디오에 추가
for image in images:
    frame = cv2.imread(os.path.join(image_folder, image))
    video.write(frame)

# 비디오 파일 저장 종료
video.release()
cv2.destroyAllWindows()